In [1]:
import requests as http
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from lib.lastfm import Lastfm
from lib.spotify import User
import time
import calendar
import datetime
pd.options.display.max_rows = 20
api_key = 'b333a19b2c0397e8e4c1224b49b3e7cd'
api_base = 'http://ws.audioscrobbler.com/2.0/?format=json&'
user_name = 'dude0faw3'
data = http.get(api_base + 'method=user.getrecenttracks&limit=200&api_key='\
                + api_key + '&user=' + user_name).json()

In [2]:
token = 'BQBOGG9-qvFcz-zNlDxICZKLUJejNN6UyYexOPsrZNfYwGg9heoKfShskjm78J3EGYTAp93SX2GHveR5b7p51Yy7i3TkuTiNdIR-f8mdSdo1I4uWChEZ8OuwhRnQR5hIvTz4dWDoPbMorNVTf4p5FzzXzwF4_Mvpom3autpp4E3rOpKhJjLRBF150iE43u82RqqTp6gLkBGE'
received_features = ['danceability', 'energy', 'acousticness', 'valence', 'tempo']
user = User(received_features, 40, token=token)
lastfm = Lastfm()

Jewels & Dreams
Sultan + Shepard 
Meg & Dia
Jenna G and Futurebound
T.I. & Playboy Tre


In [3]:
track_data = data['recenttracks']['track']
limit = data['recenttracks']['@attr']['totalPages']
page = 1
history = []
# for nested dictionary entry requests
class FakeNone:
    def get(*args):
        return FakeNone()
# ensures that there's no key error exception
def check_track(track):
    arr = [track.get('name', FakeNone()),
     track.get('artist', FakeNone()).get('#text', FakeNone()),
      track.get('date', FakeNone()).get('uts', FakeNone())
    ]
    return [None if type(x) == FakeNone else x for x in arr]

for track in track_data:
    history.append(check_track(track))
    
print('Starting! There are ' + limit + ' pages')
while int(data['recenttracks']['@attr']['page']) is not int(limit):
    if page % 25 is 0:
        print('Page %i' % page)
    page += 1
    data = http.get(api_base + 'method=user.getrecenttracks&limit=200&api_key=' + api_key + \
                    '&user=' + user_name + '&page=' + str(page)).json()
    for track in data['recenttracks']['track']:
        history.append(check_track(track))



Starting! There are 157 pages
Page 25
Page 50
Page 75
Page 100
Page 125
Page 150


## Goals:
- most listened to songs / artists (as well as during a time)
- songs/artists you've neglected
- genres (tags)

In [4]:
history_df = pd.DataFrame(history)
history_df.columns = ['track_name', 'artists', 'date']

In [5]:
# determine the n-most frequently played songs from a given pandas series
def determine_frequencies(data, num_songs, neglected=False):
    if neglected:
        counter = Counter(data)
        frequencies = counter.most_common()
        index = len(frequencies) - num_songs
        return frequencies[ index :]
    else:
        counter = Counter(data)
        return counter.most_common(num_songs)

# clean lastfm data to only have songs currently in spotify library
def refine_df(original_df, feature_name, must_haves):
    def check(val):
        for must_have in must_haves:
            if must_have.lower() == val.lower():
                return True
            
        return False
    arr_check = np.vectorize(check)
    return original_df[ arr_check(original_df[ feature_name ]) ]

def set_count(spotify, history):
    history_lowercase = history['track_name'].apply(lambda x: x.lower())
    counter = Counter(history_lowercase)
    total_count = dict(counter.most_common())
    count_arr = []
    spotify_lowercase = spotify['track_name'].apply(lambda x: x.lower())
    for track in spotify_lowercase:
        try:
            count_arr.append(total_count[track])
        except KeyError:
            count_arr.append(0)
    spotify['count'] = pd.Series(count_arr)
        
songs = user.library['track_name']
history_df = refine_df(history_df, 'track_name', songs)
set_count(user.library, history_df )

In [6]:
user.library

,track_name,track_id,artists,popularity,genres,danceability,energy,acousticness,valence,tempo,count
0,Febreze (feat. 2 Chainz),48bSfSZaq9Aizbu4AWn4st,"Jack Ü,Skrillex,Diplo,2 Chainz",63,"electronic,trap,dubstep,electronic,electronic,...",0.680,0.983,0.018400,0.292,149.829,155
1,Losin Control,1jkIErXa3YNUX5QIyO6GGR,Russ,75,"rap,underlisteners",0.761,0.250,0.558000,0.249,96.992,1
2,Limelight (NGHTMRE Remix),3mof6Z6vz6gonsuIEQXank,"Just A Gent,NGHTMRE",60,"electronic,dubstep,trap,electronica",0.574,0.576,0.057900,0.285,145.094,10
3,Enigma (feat. GRRL PAL),5wd14CMjlqbSjibnpoTgla,"SLUMBERJACK,GRRL PAL",54,"electronic,chillout,pop,electronic",0.631,0.909,0.006830,0.636,153.931,4
4,Rebel,0htTZnlk6okQ1HIq4EvFQ6,G-Eazy,54,"hiphop,rap",0.682,0.911,0.143000,0.554,140.079,2
5,The Others (feat. KLP) - Nghtmre Remix,5M0DX04ViGG3T1NXiB9EOu,"SLUMBERJACK,KLP,NGHTMRE",33,"electronic,chillout,pop,australian,trap,electr...",0.610,0.831,0.006040,0.430,102.001,14
6,Miracle,7p9jHBSk1pOciORji5sPV8,"Borgeous,BRKLYN,Lenachka",35,"electronic,house,electronic,,pop,indie",0.468,0.776,0.006400,0.217,127.982,38
7,Wanna Lose You,5Hfi7ZlAERXAvqmXUrVYa2,"Borgeous,tyDi",48,"electronic,house,trance,progressivetrance",0.550,0.794,0.101000,0.463,159.923,8
8,Ride It,2HrYDWxXQKIvOvwS293bX4,"Borgeous,Rvssian,M.R.I.,Sean Paul",48,"electronic,house,,,electronic,house,dancehall,...",0.749,0.702,0.159000,0.357,130.051,16
9,Tsunami - Original Mix,3felG0BIvmmOoCIeE2A5vq,"DVBBS,Borgeous",56,"house,electronic,electronic,house",0.698,0.838,0.049000,0.289,128.004,6


In [7]:
class TimeMachine:
    
    def __init__(self, year):
        self.seasons = {
            'spring': ( time.strptime('21 Mar ' + year, '%d %b %Y'),
                      time.strptime('21 Jun ' + year, '%d %b %Y')),
            'summer': ( time.strptime('21 Jun ' + year, '%d %b %Y'),
                      time.strptime('23 Sep ' + year, '%d %b %Y')),
            'fall': ( time.strptime('23 Sep ' + year, '%d %b %Y'),
                      time.strptime('21 Dec ' + year, '%d %b %Y')),
            'winter': ( time.strptime('21 Dec ' + year, '%d %b %Y'),
                        time.strptime('21 Mar ' + str(int(year) + 1), '%d %b %Y'))
        }
        self.year = year
        
    def set_year(self, new_year):
        self = self.__init__(new_year)
        
    def in_season(self, season, df):
        timeslice = self.seasons[season]
        def check_time(seconds):
            try:
                listen_time =  time.gmtime(int(seconds))
            except TypeError:
                return False
            return listen_time > timeslice[0] and listen_time < timeslice[1]
        arr_time_check = np.vectorize(check_time)
        return df[arr_time_check(df['date'])]
    
    def in_year(self, df):
        date = datetime.date(int(self.year), 1, 1)
        def check_time(seconds):
            try: 
                listen_time =  datetime.date.fromtimestamp(int(seconds))
            except TypeError:
                return False
            return listen_time.year == date.year
        arr_time_check = np.vectorize(check_time)
        return df[ arr_time_check(df['date'])]
    


In [8]:
# make a timemachine to 2014, check out top artists from then
obj = TimeMachine('2014')
year_df = obj.in_year(history_df)
determine_frequencies(year_df['artists'], 20)

[('Blackmill', 130),
 ('Eminem', 128),
 ('Calvin Harris', 126),
 ('OneRepublic', 89),
 ('The Glitch Mob', 78),
 ('Bassnectar', 75),
 ('PANTyRAiD', 75),
 ('Doctor P', 68),
 ('Flux Pavilion', 68),
 ('The Script', 65),
 ('Maroon 5', 63),
 ('Linkin Park', 63),
 ('Hardwell', 60),
 ('Nero', 60),
 ('Lifehouse', 53),
 ('Major Lazer', 47),
 ('Armin van Buuren', 46),
 ('The Fray', 45),
 ('Rogue', 43),
 ('Bare', 43)]

In [9]:
user.library.head()

,track_name,track_id,artists,popularity,genres,danceability,energy,acousticness,valence,tempo,count
0,Febreze (feat. 2 Chainz),48bSfSZaq9Aizbu4AWn4st,"Jack Ü,Skrillex,Diplo,2 Chainz",63,"electronic,trap,dubstep,electronic,electronic,...",0.680,0.983,0.01840,0.292,149.829,155
1,Losin Control,1jkIErXa3YNUX5QIyO6GGR,Russ,75,"rap,underlisteners",0.761,0.250,0.55800,0.249,96.992,1
2,Limelight (NGHTMRE Remix),3mof6Z6vz6gonsuIEQXank,"Just A Gent,NGHTMRE",60,"electronic,dubstep,trap,electronica",0.574,0.576,0.05790,0.285,145.094,10
3,Enigma (feat. GRRL PAL),5wd14CMjlqbSjibnpoTgla,"SLUMBERJACK,GRRL PAL",54,"electronic,chillout,pop,electronic",0.631,0.909,0.00683,0.636,153.931,4
4,Rebel,0htTZnlk6okQ1HIq4EvFQ6,G-Eazy,54,"hiphop,rap",0.682,0.911,0.14300,0.554,140.079,2


In [10]:
common_genres = [
    'hiphop',
    'rap',
    'pop',
    'rock',
    'rnb',
    'dubstep',
    'trance',
    'house',
    'trap',
    'country',
    'alternative',
    'indie',
    'reggae'
]

library_genres = user.library['genres']
def count_appearances(match_genre):
    count = 0
    for track_genre in library_genres:
        track_genre = track_genre.split(',')
        for genre in track_genre:
            if genre == match_genre:
                count += 1
    return count

count = [count_appearances(genre) for genre in common_genres]
for common_genre, genre_count in zip(common_genres, count):
    print(common_genre, genre_count)

hiphop 237
rap 173
pop 95
rock 73
rnb 76
dubstep 208
trance 35
house 102
trap 43
country 1
alternative 18
indie 32
reggae 10


In [11]:
def filter_for_genre(genre, df):
    truth_arr = [False] * len(df)
    for index, genres in enumerate(df['genres']):
        genres = genres.split(',')
        if genre in genres:
            truth_arr[index] = True
    return df[truth_arr]

filter_for_genre('rap', user.library)

,track_name,track_id,artists,popularity,genres,danceability,energy,acousticness,valence,tempo,count
0,Febreze (feat. 2 Chainz),48bSfSZaq9Aizbu4AWn4st,"Jack Ü,Skrillex,Diplo,2 Chainz",63,"electronic,trap,dubstep,electronic,electronic,...",0.680,0.983,0.01840,0.2920,149.829,155
1,Losin Control,1jkIErXa3YNUX5QIyO6GGR,Russ,75,"rap,underlisteners",0.761,0.250,0.55800,0.2490,96.992,1
4,Rebel,0htTZnlk6okQ1HIq4EvFQ6,G-Eazy,54,"hiphop,rap",0.682,0.911,0.14300,0.5540,140.079,2
11,Who Knew,2AqGzt3g1L447rpkfqByiR,Lil Dicky,56,"hiphop,rap",0.755,0.787,0.32500,0.5720,91.026,4
15,Same Old Song,3nE2A8sGDAI2nzGSI8Rzd6,"The Weeknd,Juicy J",35,"rnb,electronic,hiphop,rap",0.746,0.565,0.38700,0.3520,109.959,22
18,The Zone - Album Version (Edited),5zggls08Fw43MEb2r9kSUW,"The Weeknd,Drake",37,"rnb,electronic,hiphop,rap",0.608,0.525,0.27500,0.2670,139.978,9
28,Ni**as In Paris,4Li2WHPkuyCdtmokzW2007,"JAY Z,Kanye West",75,"hiphop,rap,hiphop,rap",0.788,0.859,0.12700,0.7650,140.027,0
30,Seduction,7tOqYtca4gJRg1qYZZ36Mc,Eminem,29,"rap,hiphop",0.603,0.833,0.11600,0.4710,77.956,5
31,Fade Away,4QRWA5UqYU1st9aO0UfxOj,Logic,68,"hiphop,rap",0.488,0.774,0.26300,0.5220,74.893,3
32,A Word From Our Sponsor,0BO1a9WQ1qCwORuIkFKjjC,Logic,31,"hiphop,rap",0.699,0.461,0.81600,0.6980,128.447,12


In [12]:
determine_frequencies(history_df['track_name'].values, 10, neglected=True)

[('Tumblr Girls', 3),
 ('The Distance', 2),
 ('We Come Running - Tiësto Remix', 2),
 ('Rebel', 2),
 ('Many Ways - Radio Edit', 2),
 ('Downtown Love', 2),
 ('Valerie', 2),
 ('Flexicution', 2),
 ('bring me to life', 1),
 ('Losin Control', 1)]

## References
Stéfan van der Walt, S. Chris Colbert and Gaël Varoquaux. The NumPy Array: A Structure for Efficient Numerical Computation, Computing in Science & Engineering, 13, 22-30 (2011)

John D. Hunter, "Matplotlib: A 2D Graphics Environment", Computin

Raschka, Sebastian, and Randal S. Olson. Python Machine Learning: Unlock Deeper Insights into Machine    Learning with This Vital Guide to Cutting-edge Predictive Analytics. Birmingham: Packt, 2015. Print.g in Science & Engineering, vol.9, no. 3, pp. 90-95, May/June 2007, doi:10.1109/MCSE.2007.55  